In [81]:
import pyaudio
import pylab
import os
from struct import pack
from sys import byteorder
import numpy as np
import matplotlib.pyplot as plt
from PyQt5 import QtWidgets, QtCore
import PyQt5 as pg
import scipy
from scipy.signal import find_peaks
from scipy.fftpack import fft
import time
from tkinter import TclError
from scipy.io import wavfile as wav
import wave
from pydub import AudioSegment
import math
from collections import Counter
from array import array
from pydub.silence import split_on_silence
import datetime
import crepe
import copy



In [4]:
# White keys are in Uppercase and black keys (sharps) are in lowercase
octave = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B'] 
base_freq = 440 #Frequency of Note A4
keys = np.array([x+str(y) for y in range(0,9) for x in octave])
# Trim to correct # keys
start = np.where(keys == 'A0')[0][0]
end = np.where(keys == 'C8')[0][0]
keys = keys[start:end+1]

note_freqs = list(([2**((n-48)/12)*base_freq for n in range(len(keys))]))

In [5]:
c_major_scale_freqs=[]
c_major_scale=['C4', 'D4', 'E4', 'F4', 'G4']
for i in range(len(keys)):
    for j in range(5):
        if c_major_scale[j]==keys[i]:
            c_major_scale_freqs.append(note_freqs[i])
print(c_major_scale_freqs)

[261.6255653005986, 293.6647679174076, 329.6275569128699, 349.2282314330039, 391.99543598174927]


In [6]:
def closest_value(input_list, input_value):

  arr = np.asarray(input_list)

  i = (np.abs(arr - input_value)).argmin()

  return arr[i]

In [90]:
# constants
CHUNK = 1024*4          # samples per frame
FORMAT = pyaudio.paInt16     # audio format (bytes per sample?)
CHANNELS = 1                 # single channel for microphone
RATE = 48000                 # samples per second
RECORD_SECONDS = 1
THRESHOLD=500
TRIM_APPEND = RATE / 4
FRAME_MAX_VALUE = 2 ** 15 - 1
NORMALIZE_MINUS_ONE_dB = 10 ** (-1.0 / 20)
SILENT_CHUNKS = (3 * RATE / CHUNK)  # about 1sec




In [151]:
p = pyaudio.PyAudio()
p.get_device_info_by_index(3)['defaultSampleRate']

48000.0

In [88]:
def is_silent(snd_data):
    "Returns 'True' if below the 'silent' threshold"
    return max(snd_data) < THRESHOLD

def normalize(data_all):
    """Amplify the volume out to max -1dB"""
    # MAXIMUM = 16384
    normalize_factor = (float(NORMALIZE_MINUS_ONE_dB * FRAME_MAX_VALUE)
                        / max(abs(i) for i in data_all))

    r = array('h')
    for i in data_all:
        r.append(int(i * normalize_factor))
    return r
def trim(data_all):
    _from = 0
    _to = len(data_all) - 1
    for i, b in enumerate(data_all):
        if abs(b) > THRESHOLD:
            _from = max(0, i - TRIM_APPEND)
            break

    for i, b in enumerate(reversed(data_all)):
        if abs(b) > THRESHOLD:
            _to = min(len(data_all) - 1, len(data_all) - 1 - i + TRIM_APPEND)
            break

    return copy.deepcopy(data_all[int(_from):(int(_to) + 1)])

In [91]:
def record():
    """Record a word or words from the microphone and 
    return the data as an array of signed shorts."""

    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, output=True, frames_per_buffer=SILENT_CHUNKS)

    silent_chunks = 0
    audio_started = False
    data_all = array('h')

    while True:
        # little endian, signed short
        data_chunk = array('h', stream.read(CHUNK))
        if byteorder == 'big':
            data_chunk.byteswap()
        data_all.extend(data_chunk)

        silent = is_silent(data_chunk)

        if audio_started:
            if silent:
                silent_chunks += 1
                if silent_chunks > SILENT_CHUNKS:
                    break
            else: 
                silent_chunks = 0
        elif not silent:
            audio_started = True              

    sample_width = p.get_sample_size(FORMAT)
    stream.stop_stream()
    stream.close()
    p.terminate()

    data_all = trim(data_all)  # we trim before normalize as threshhold applies to un-normalized wave (as well as is_silent() function)
    data_all = normalize(data_all)
    return sample_width, data_all

def record_to_file(path):
    "Records from the microphone and outputs the resulting data to 'path'"
    sample_width, data = record()
    data = pack('<' + ('h' * len(data)), *data)

    wave_file = wave.open(path, 'wb')
    wave_file.setnchannels(CHANNELS)
    wave_file.setsampwidth(sample_width)
    wave_file.setframerate(RATE)
    wave_file.writeframes(data)
    wave_file.close()

record_to_file('/Users/emmarobinson/Downloads/piano_audio_new.wav')

TypeError: integer argument expected, got float

In [78]:
#os.remove('/Users/emmarobinson/Downloads/piano_audio.wav') #change this according to your computer file
#os.remove('/Users/emmarobinson/Downloads/piano_audio_new.wav') #change this according to your computer file
# create matplotlib figure and axes
#fig, (ax1, ax2) = plt.subplots(2, figsize=(15, 7))

# pyaudio class instance

p = pyaudio.PyAudio()
#p.get_device_info_by_index(3)['defaultSampleRate']

# stream object to get data from microphone
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input_device_index =3, #change this according to the mic port on your computer
    input=True,
    frames_per_buffer=CHUNK
)
frames = []
print("* listening")

    

#if vol >= 400:


while 1:
    
    data = stream.read(CHUNK, exception_on_overflow = False)
    data_chunk=array('h',data)
    vol=max(data_chunk)
    if vol >= 400:
        print("* recording")
        for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
            stream.write(data, CHUNK)
            frames.append(data)
        

    else:
        print("* waiting")

print("* done")

stream.stop_stream()
stream.close()

p.terminate()
file_path='/Users/emmarobinson/Downloads/piano_audio.wav' #change this according to your computer file
file_name = file_path.split('/')[-1]
wf = wave.open(file_path, "wb")
# set the channels
wf.setnchannels(CHANNELS)
# set the sample format
wf.setsampwidth(p.get_sample_size(FORMAT))
# set the sample rate
wf.setframerate(RATE)
# write the frames as bytes
wf.writeframes(b"".join(frames))
# close the file
wf.close()


* listening
* waiting
* waiting
* waiting
* waiting
* waiting
* waiting
* waiting
* waiting
* waiting
* waiting
* waiting
* waiting
* waiting
* waiting
* waiting
* waiting
* waiting
* waiting
* waiting
* waiting
* waiting
* waiting
* recording


OSError: [Errno Not output stream] -9974

In [75]:
sound = AudioSegment.from_file(file_path, format = 'wav') 
audio_chunks = split_on_silence(sound
                            ,min_silence_len = 100
                            ,silence_thresh = -45
                            ,keep_silence = 50
                        )

# Putting the file back together
combined = AudioSegment.empty()
for chunk in audio_chunks:
    combined += chunk
combined.export(f'/Users/emmarobinson/Downloads/piano_audio_new.wav', format = 'wav') #change this according to your computer file

<_io.BufferedRandom name='/Users/emmarobinson/Downloads/piano_audio_new.wav'>

In [76]:
sr, audio = wav.read('/Users/emmarobinson/Downloads/piano_audio_new.wav') #change this according to your computer file
time, frequency, confidence, activation = crepe.predict(audio, sr, viterbi=True)
freq1=np.array([np.mean(frequency)])

18/18 [==============================] - 29s 2s/step


In [77]:
for i in range(len(freq1)):
    note=closest_value(note_freqs,freq1[i])
    index = note_freqs.index(note)
    print(keys[index])

D#4


In [ ]:
for i in range(len(freq1)):
    note=closest_value(note_freqs,freq1[i])
    index = note_freqs.index(note)
    played_note=keys[index]

x=0
if played_note==c_major_scale[x]:
    print("yay")
else:
        print("try again")

In [155]:
rate, data = wav.read('/Users/emmarobinson/Downloads/piano_audio_new.wav') #change this according to your computer file

audio_segment = AudioSegment.from_file('/Users/emmarobinson/Downloads/piano_audio_new.wav') #change this according to your computer file

duration = len(audio_segment)/1000

# calculate the length of our chunk in the np.array using sample rate
chunk = int(rate * duration)

# length of delimiting 1600hz tone
offset = int(rate * 0.005)

# number of bits in the audio data to decode
bits = int(len(data) / chunk)

def get_freq(bit):
    # start position of the current bit
    strt = (chunk * bit) 
    
    # remove the delimiting 1600hz tone
    end = (strt + chunk) 
   
    
    # slice the array for each bit
    sliced = data[strt:end]

    w = np.fft.fft(sliced)
    w=np.absolute(w)
    freqs = np.fft.fftfreq(len(w))

    

    # Find the peak in the coefficients
    idx = np.argmax(np.abs(w))
    freq = freqs[idx]
    freq_in_hertz = abs(freq * rate)
    return freq_in_hertz

decoded_freqs = [get_freq(bit) for bit in range(bits)]

In [31]:
'''
# variable for plotting
x = np.arange(0, 2 * CHUNK, 2)       # samples (waveform)
xf = np.linspace(0, RATE, CHUNK)     # frequencies (spectrum)

# create a line object with random data
line, = ax1.plot(x, np.random.rand(CHUNK), '-', lw=2)

# create semilogx line for spectrum
line_fft, = ax2.semilogx(xf, np.random.rand(CHUNK), '-', lw=2)

# format waveform axes
ax1.set_title('AUDIO WAVEFORM')
ax1.set_xlabel('samples')
ax1.set_ylabel('volume')
ax1.set_ylim(0, 255)
ax1.set_xlim(0, 2 * CHUNK)
plt.setp(ax1, xticks=[0, CHUNK, 2 * CHUNK], yticks=[0, 128, 255])

# format spectrum axes
ax2.set_xlim(20, RATE / 2)

print('stream started')

# for measuring frame rate
frame_count = 0
start_time = time.time()

while True:
    
    # binary data
    data = stream.read(CHUNK,exception_on_overflow = False)  
    
    # convert data to integers, make np array, then offset it by 127
    data_int = np.frombuffer(data, dtype='h')  
    data_np = np.array(data_int, dtype='h')/140 + 255
    
    line.set_ydata(data_np)
    
    # compute FFT and update line
    yf = fft(data_int)
    line_fft.set_ydata(np.abs(yf[0:CHUNK])  / (128 * CHUNK))
    
    # update figure canvas
    try:
        fig.canvas.draw()
        fig.canvas.flush_events()
        frame_count += 1
        
    except TclError:
                # calculate average frame rate
        frame_rate = frame_count / (time.time() - start_time)
        
        print('stream stopped')
        print('average frame rate = {:.0f} FPS'.format(frame_rate))
        break
    '''

NameError: name 'ax1' is not defined